In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pathlib

# custom package(s)
from utils import utils, synapse_login

import yaml
import pathlib

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

csv_model = pathlib.Path("../" + config["file_names"]["csv_model"]).resolve()
json_model = pathlib.Path("../" + config["file_names"]["json_model"]).resolve()

In [12]:
dm_name = "../EL.data.model.csv"
output_dir = pathlib.Path("../backups").resolve()

dm_path = pathlib.Path(dm_name).resolve()
dm = utils.load_and_backup_dm(dm_path, output_dir)
# dm = dm.drop(columns=["Unnamed: 0"])

In [61]:
# dm = pd.read_csv('../backups/EL.data.model-2023-10-12.15-02.csv')

In [66]:
# clean up attributes
# remove any special characters
pattern = r"\(|\)|\.|-|_|:|;"
dm[['Attribute', 'DependsOn', 'Valid Values']] = dm[['Attribute', 'DependsOn', 'Valid Values']
                                                    ].apply(lambda x: x.str.replace(pattern, '').str.strip().str.replace('\s+', ''), axis=1)

In [24]:
dm['Attribute'] = dm['Attribute'].str.strip('"')

In [27]:
dm['Properties'].unique()

array(['ValidValue', 'unspecified', 'DataProperty', 'DataType',
       'MetadataType', 'DataSubtype'], dtype=object)

In [28]:
dm['Parent'] = dm['Parent'].str.replace('Template', 'DataType')

In [39]:
dm.loc[dm['Parent'] == 'MeasurementUnit'].to_csv('./measurementUnits.csv')

In [43]:
dm = dm.drop(index=dm.loc[dm['Parent'] == 'MeasurementUnit'].index).reset_index(drop = True)

In [49]:
dm[(dm['Attribute'].str.contains('units', flags = re.IGNORECASE)) & (~dm['Valid Values'].isna())]

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue,UsedIn,module
370,conversionRatioUnits,The units of measurement.,"AFU,AI,AU/ml,DKunits/ml,HAU,IU,IU/ml,Kallikrei...",True,NaN,NaN,unspecified,NaN,bsSeq,unspecified,Sage Bionetworks,STRING,Sage Bionetworks,False,NaN,NaN
484,readLengthUnits,The units of measurement.,"AFU,AI,AU/ml,DKunits/ml,HAU,IU,IU/ml,Kallikrei...",True,NaN,NaN,unspecified,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe...",unspecified,Sage Bionetworks,STRING,Sage Bionetworks,False,NaN,NaN


In [54]:
dm['Properties'] = dm['Properties'].str.replace('DataType', '')

In [10]:
# dm = dm[dm['Properties'] != 'ValidValue'].sort_values(by='Attribute').reset_index(drop = True)

In [11]:
# write out new model
dm.to_csv("../EL.data.model.csv", index=False)

In [68]:
# convert csv model to jsonld
!schematic schema convert {csv_model} --output_jsonld {json_model}

Starting schematic...
Done adding requirements and value ranges to attributes
The Data Model was created and saved to '/Users/nlee/Documents/Projects/ELITE/ELITE-data-models/EL.data.model.jsonld' location.
